In [91]:
import pandas as pd
import re
import numpy as np

In [92]:
times = pd.read_csv('timesData.csv', sep= ',')
shangai = pd.read_csv('shanghaiData.csv')
cwur = pd.read_csv('cwurData.csv')
attainment = pd.read_csv('educational_attainment_supplementary_data.csv')

In [93]:
times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2603 non-null   object 
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2603 non-null   object 
 8   total_score             2603 non-null   object 
 9   num_students            2544 non-null   object 
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   object 
 12  female_male_ratio       2370 non-null   object 
 13  year                    2603 non-null   int64  
dtypes: float64(4), int64(1), object(9)
memor

In [94]:
cwur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            2200 non-null   int64  
 1   institution           2200 non-null   object 
 2   country               2200 non-null   object 
 3   national_rank         2200 non-null   int64  
 4   quality_of_education  2200 non-null   int64  
 5   alumni_employment     2200 non-null   int64  
 6   quality_of_faculty    2200 non-null   int64  
 7   publications          2200 non-null   int64  
 8   influence             2200 non-null   int64  
 9   citations             2200 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2200 non-null   int64  
 12  score                 2200 non-null   float64
 13  year                  2200 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 240.8+ KB


In [95]:
shangai.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4897 entries, 0 to 4896
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   world_rank       4897 non-null   object 
 1   university_name  4896 non-null   object 
 2   national_rank    4896 non-null   object 
 3   total_score      1101 non-null   float64
 4   alumni           4896 non-null   float64
 5   award            4895 non-null   float64
 6   hici             4895 non-null   float64
 7   ns               4875 non-null   float64
 8   pub              4895 non-null   float64
 9   pcp              4895 non-null   float64
 10  year             4897 non-null   int64  
dtypes: float64(7), int64(1), object(3)
memory usage: 421.0+ KB


In [96]:
cwur=cwur.rename(columns={"institution":"university_name"})

In [97]:
times['female_male_ratio'] = times['female_male_ratio'].astype(str)

In [98]:
times['num_students'] =times['num_students'].str.replace(',',"").astype(float)

In [99]:
times['income']= times['income'].apply(pd.to_numeric, errors= 'coerce')
times

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,nan,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,7929.0,8.4,27%,45 : 55,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,-,21958.0,15.3,3%,48 : 52,2016
2599,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,-,31268.0,28.7,2%,36 : 64,2016
2600,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,-,4122.0,3.7,3%,nan,2016
2601,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,-,10117.0,12.1,8%,28 : 72,2016


## 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [100]:
dataframe1 = times.iloc[times.groupby('university_name').idxmax()['year']]

dataframe1

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,20%,46 : 54,2016
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,2218.0,8.0,14%,49 : 51,2016
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,18 : 82,2016
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,9666.0,10.5,54%,27 : 73,2016


In [11]:
dataframe2 = times.iloc[times.groupby('university_name').idxmin()['year']]
dataframe2

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,17422.0,15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,16099.0,24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,23895.0,13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,9252.0,19.2,18%,48 : 52,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,2400.0,7.9,20%,46 : 54,2011
99,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,2218.0,8.0,14%,49 : 51,2011
38,39,École Polytechnique,France,57.9,77.9,56.1,91.4,NaN,69.5,2429.0,4.8,30%,18 : 82,2011
47,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,9666.0,10.5,54%,27 : 73,2011


## 2. For each university, compute the improvement in income between the least recent and the most recent data points

In [12]:
df= times.iloc[times.groupby('university_name').idxmax()['year']][['university_name','year','income']]
df

,university_name,year,income
2405,AGH University of Science and Technology,2016,NaN
2003,Aalborg University,2016,43.7
2056,Aalto University,2016,61.6
1908,Aarhus University,2016,68.3
2105,Aberystwyth University,2016,31.3
...,...,...,...
1856,École Normale Supérieure,2016,37.1
2013,École Normale Supérieure de Lyon,2016,31.7
1904,École Polytechnique,2016,82.3
1833,École Polytechnique Fédérale de Lausanne,2016,65.4


In [13]:
df2= times.iloc[times.groupby('university_name').idxmin()['year']][['university_name','year','income']]
df2

,university_name,year,income
2405,AGH University of Science and Technology,2016,NaN
501,Aalborg University,2012,36.4
502,Aalto University,2012,61.9
166,Aarhus University,2011,61.5
476,Aberystwyth University,2012,35.5
...,...,...,...
41,École Normale Supérieure,2011,30.7
99,École Normale Supérieure de Lyon,2011,26.1
38,École Polytechnique,2011,NaN
47,École Polytechnique Fédérale de Lausanne,2011,38.0


In [14]:
df3 = pd.merge(df, df2, on= 'university_name')
df3.head(20)

,university_name,year_x,income_x,year_y,income_y
0,AGH University of Science and Technology,2016,NaN,2016,NaN
1,Aalborg University,2016,43.7,2012,36.4
2,Aalto University,2016,61.6,2012,61.9
3,Aarhus University,2016,68.3,2011,61.5
4,Aberystwyth University,2016,31.3,2012,35.5
5,Adam Mickiewicz University,2016,28.7,2016,28.7
6,Aix-Marseille University,2016,33.1,2016,33.1
7,Ajou University,2016,45.7,2016,45.7
8,Alexandria University,2016,29.7,2011,36.0
9,Alexandru Ioan Cuza University,2016,28.2,2016,28.2


In [15]:
df3['improvement'] = abs(df3['income_x'] - df3['income_y'])
df3

,university_name,year_x,income_x,year_y,income_y,improvement
0,AGH University of Science and Technology,2016,NaN,2016,NaN,NaN
1,Aalborg University,2016,43.7,2012,36.4,7.3
2,Aalto University,2016,61.6,2012,61.9,0.3
3,Aarhus University,2016,68.3,2011,61.5,6.8
4,Aberystwyth University,2016,31.3,2012,35.5,4.2
...,...,...,...,...,...,...
813,École Normale Supérieure,2016,37.1,2011,30.7,6.4
814,École Normale Supérieure de Lyon,2016,31.7,2011,26.1,5.6
815,École Polytechnique,2016,82.3,2011,NaN,NaN
816,École Polytechnique Fédérale de Lausanne,2016,65.4,2011,38.0,27.4


## 3. Find the university with the largest increase computed in the previous point

In [16]:
df3[df3['improvement']== df3['improvement'].max()]  # abbiamo deciso di lasciare i nan e non forzarli con gli zero, 
#perchè la colonna income potrebbe essere qualunque cosa, e i valori poterebbero essere diversi da zero. 

,university_name,year_x,income_x,year_y,income_y,improvement
428,TU Dresden,2016,99.7,2012,31.9,67.8


## 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [17]:
time= times.iloc[times.groupby('university_name').idxmax()['year']][['world_rank','university_name','year']]
time['world_rank_in']= time['world_rank'].str.extract('(\d+)').astype(int)
time['world_rank_fi']= time['world_rank'].str.extract('(\d+$)').astype(int)
time

,world_rank,university_name,year,world_rank_in,world_rank_fi
2405,601-800,AGH University of Science and Technology,2016,601,800
2003,201-250,Aalborg University,2016,201,250
2056,251-300,Aalto University,2016,251,300
1908,=106,Aarhus University,2016,106,106
2105,301-350,Aberystwyth University,2016,301,350
...,...,...,...,...,...
1856,54,École Normale Supérieure,2016,54,54
2013,201-250,École Normale Supérieure de Lyon,2016,201,250
1904,=101,École Polytechnique,2016,101,101
1833,31,École Polytechnique Fédérale de Lausanne,2016,31,31


In [18]:
time['mean']=(time['world_rank_in']+time['world_rank_fi'])//2
time

,world_rank,university_name,year,world_rank_in,world_rank_fi,mean
2405,601-800,AGH University of Science and Technology,2016,601,800,700
2003,201-250,Aalborg University,2016,201,250,225
2056,251-300,Aalto University,2016,251,300,275
1908,=106,Aarhus University,2016,106,106,106
2105,301-350,Aberystwyth University,2016,301,350,325
...,...,...,...,...,...,...
1856,54,École Normale Supérieure,2016,54,54,54
2013,201-250,École Normale Supérieure de Lyon,2016,201,250,225
1904,=101,École Polytechnique,2016,101,101,101
1833,31,École Polytechnique Fédérale de Lausanne,2016,31,31,31


In [19]:
shang= shangai.iloc[shangai.groupby('university_name').idxmax()['year']][['world_rank','university_name','year']]
shang['world_rank_in']= shang['world_rank'].str.extract('(\d+)').astype(int)
shang['world_rank_fi']= shang['world_rank'].str.extract('(\d+$)').astype(int)
shang

,world_rank,university_name,year,world_rank_in,world_rank_fi
4697,301-400,Aalborg University,2015,301,400
4797,401-500,Aalto University,2015,401,500
4469,73,Aarhus University,2015,73,73
4497,101-150,Aix Marseille University,2015,101,150
3115,102-150,Aix-Marseille University,2011,102,150
...,...,...,...,...,...
3512,401-500,Yamaguchi University,2011,401,500
4695,201-300,Yeshiva University,2015,201,300
4696,201-300,Yonsei University,2015,201,300
4396,401-500,York University,2014,401,500


In [20]:
shang['mean']=(shang['world_rank_in']+shang['world_rank_fi'])//2
shang

,world_rank,university_name,year,world_rank_in,world_rank_fi,mean
4697,301-400,Aalborg University,2015,301,400,350
4797,401-500,Aalto University,2015,401,500,450
4469,73,Aarhus University,2015,73,73,73
4497,101-150,Aix Marseille University,2015,101,150,125
3115,102-150,Aix-Marseille University,2011,102,150,126
...,...,...,...,...,...,...
3512,401-500,Yamaguchi University,2011,401,500,450
4695,201-300,Yeshiva University,2015,201,300,250
4696,201-300,Yonsei University,2015,201,300,250
4396,401-500,York University,2014,401,500,450


In [21]:
cw= cwur.iloc[cwur.groupby('university_name').idxmax()['year']][['world_rank','university_name','year']]
cw['world_rank']= cw['world_rank'].astype(str)
cw['world_rank_in']= cw['world_rank'].str.extract('(\d+)').astype(int)
cw['world_rank_fi']= cw['world_rank'].str.extract('(\d+$)').astype(int)
cw

,world_rank,university_name,year,world_rank_in,world_rank_fi
1981,782,AGH University of Science and Technology,2015,782,782
1764,565,Aalborg University,2015,565,565
1620,421,Aalto University,2015,421,421
1321,122,Aarhus University,2015,122,122
2013,814,Aberystwyth University,2015,814,814
...,...,...,...,...,...
2080,881,École centrale de Lyon,2015,881,881
1236,37,École normale supérieure - Paris,2015,37,37
1920,721,École normale supérieure de Cachan,2015,721,721
1670,471,École normale supérieure de Lyon,2015,471,471


In [22]:
cw['mean']=(cw['world_rank_in']+cw['world_rank_fi'])//2
cw

,world_rank,university_name,year,world_rank_in,world_rank_fi,mean
1981,782,AGH University of Science and Technology,2015,782,782,782
1764,565,Aalborg University,2015,565,565,565
1620,421,Aalto University,2015,421,421,421
1321,122,Aarhus University,2015,122,122,122
2013,814,Aberystwyth University,2015,814,814,814
...,...,...,...,...,...,...
2080,881,École centrale de Lyon,2015,881,881,881
1236,37,École normale supérieure - Paris,2015,37,37,37
1920,721,École normale supérieure de Cachan,2015,721,721,721
1670,471,École normale supérieure de Lyon,2015,471,471,471


In [23]:
concat1 = pd.merge(time[['university_name','mean']], shang[['university_name','mean']], on='university_name',suffixes=('_time','_shang'))
concat2 = pd.merge(concat1[['university_name','mean_time','mean_shang']], cw[['university_name','mean']], on='university_name')
concat2=concat2.rename(columns={"mean":"mean_cwur"})
concat2.head(30)

,university_name,mean_time,mean_shang,mean_cwur
0,Aalborg University,225,350,565
1,Aalto University,275,450,421
2,Aarhus University,106,73,122
3,Aix-Marseille University,275,126,206
4,Aristotle University of Thessaloniki,700,450,459
5,Arizona State University,189,93,97
6,Auburn University,550,450,549
7,Autonomous University of Barcelona,146,250,241
8,Autonomous University of Madrid,325,250,304
9,Bangor University,325,450,568


In [24]:
def max_diff(riga):
    s = riga['mean_shang']
    c = riga['mean_cwur']
    t = riga['mean_time']
    m = min(s, c, t)
    M = max(s, c, t)
    return (M - m)

In [25]:
concat2['diff_rank'] = concat2.apply(max_diff, axis=1)
concat2

,university_name,mean_time,mean_shang,mean_cwur,diff_rank
0,Aalborg University,225,350,565,340
1,Aalto University,275,450,421,175
2,Aarhus University,106,73,122,49
3,Aix-Marseille University,275,126,206,149
4,Aristotle University of Thessaloniki,700,450,459,250
...,...,...,...,...,...
357,Yale University,12,11,11,1
358,Yeshiva University,164,250,171,86
359,Yonsei University,325,250,98,227
360,York University,325,450,337,125


## 5. Consider only the most recent data point of the "times" dataset. Compute the number of male and female students for each country.

In [26]:
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,nan,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,7929.0,8.4,27%,45 : 55,2011


In [27]:
dataframe1

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,20%,46 : 54,2016
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,2218.0,8.0,14%,49 : 51,2016
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,18 : 82,2016
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,9666.0,10.5,54%,27 : 73,2016


In [28]:
dataframe1['female_%']= dataframe1['female_male_ratio'].str.extract('(\d+)').astype(float)
dataframe1

<ipython-input-28-a610b709a3a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe1['female_%']= dataframe1['female_male_ratio'].str.extract('(\d+)').astype(float)


,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_%
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,48.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,32.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,54.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,20%,46 : 54,2016,46.0
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,2218.0,8.0,14%,49 : 51,2016,49.0
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,18 : 82,2016,18.0
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,9666.0,10.5,54%,27 : 73,2016,27.0


In [29]:
dataframe1['female_stud']=(dataframe1['num_students'] *(dataframe1['female_%']/100)).round(0)
dataframe1

<ipython-input-29-30b837a26897>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe1['female_stud']=(dataframe1['num_students'] *(dataframe1['female_%']/100)).round(0)


,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_%,female_stud
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,48.0,8363.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,32.0,5152.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,54.0,12903.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016,48.0,4441.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,20%,46 : 54,2016,46.0,1104.0
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,2218.0,8.0,14%,49 : 51,2016,49.0,1087.0
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,18 : 82,2016,18.0,437.0
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,9666.0,10.5,54%,27 : 73,2016,27.0,2610.0


In [30]:
country = dataframe1.groupby('country')[['num_students','female_stud']].sum()
country

,num_students,female_stud
country,,
Argentina,108373.0,67191.0
Australia,743627.0,391736.0
Austria,134477.0,68364.0
Bangladesh,62716.0,21323.0
Belarus,29303.0,20219.0
...,...,...
Unisted States of America,0.0,0.0
United Arab Emirates,14447.0,9516.0
United Kingdom,1324842.0,711815.0


In [31]:
country['male_stud'] = country['num_students'] - country['female_stud']
country

,num_students,female_stud,male_stud
country,,,
Argentina,108373.0,67191.0,41182.0
Australia,743627.0,391736.0,351891.0
Austria,134477.0,68364.0,66113.0
Bangladesh,62716.0,21323.0,41393.0
Belarus,29303.0,20219.0,9084.0
...,...,...,...
Unisted States of America,0.0,0.0,0.0
United Arab Emirates,14447.0,9516.0,4931.0
United Kingdom,1324842.0,711815.0,613027.0


## 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [41]:
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,nan,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,7929.0,8.4,27%,45 : 55,2011


In [56]:
dataframe1['ratio'] = (dataframe1['female_%']/100)/(1-dataframe1['female_%']/100)
dataframe1

<ipython-input-56-540221aea521>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe1['ratio'] = (dataframe1['female_%']/100)/(1-dataframe1['female_%']/100)


,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_%,female_stud,ratio
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,35569.0,17.0,1%,-,2016,NaN,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,48.0,8363.0,0.923077
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,32.0,5152.0,0.470588
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,54.0,12903.0,1.173913
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016,48.0,4441.0,0.923077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,20%,46 : 54,2016,46.0,1104.0,0.851852
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,2218.0,8.0,14%,49 : 51,2016,49.0,1087.0,0.960784
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,18 : 82,2016,18.0,437.0,0.219512
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,9666.0,10.5,54%,27 : 73,2016,27.0,2610.0,0.369863


In [57]:
dataframe1[dataframe1['ratio']== np.inf]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_%,female_stud,ratio
2227,401-500,Ewha Womans University,South Korea,23.9,34.3,18.3,50.4,57.6,-,17625.0,14.4,6%,100 : 0,2016,100.0,17625.0,inf


In [58]:
dataframe1['ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)

C:\Users\Nataliya\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [59]:
ratio= dataframe1['ratio'].mean()
ratio

1.0811020920897247

In [60]:
dataframe1['university_name'].nunique

<bound method IndexOpsMixin.nunique of 2405    AGH University of Science and Technology
2003                          Aalborg University
2056                            Aalto University
1908                           Aarhus University
2105                      Aberystwyth University
                          ...                   
1856                    École Normale Supérieure
2013            École Normale Supérieure de Lyon
1904                         École Polytechnique
1833    École Polytechnique Fédérale de Lausanne
2134                           Örebro University
Name: university_name, Length: 818, dtype: object>

In [61]:
university_ratio=dataframe1[dataframe1['ratio']<=ratio]
university_ratio['university_name'].nunique

<bound method IndexOpsMixin.nunique of 2003                          Aalborg University
2056                            Aalto University
2105                      Aberystwyth University
2406                             Ajou University
2408                       Alexandria University
                          ...                   
2104                         Zhejiang University
1856                    École Normale Supérieure
2013            École Normale Supérieure de Lyon
1904                         École Polytechnique
1833    École Polytechnique Fédérale de Lausanne
Name: university_name, Length: 346, dtype: object>

## 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).


In [62]:
university_ratio

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,female_%,female_stud,ratio
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,48.0,8363.0,0.923077
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,32.0,5152.0,0.470588
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016,48.0,4441.0,0.923077
2406,601-800,Ajou University,South Korea,19.5,20.0,11.9,23.9,45.7,-,12706.0,11.3,2%,33 : 67,2016,33.0,4193.0,0.492537
2408,601-800,Alexandria University,Egypt,20.3,33.7,8.2,14.1,29.7,-,127431.0,23.3,1%,46 : 54,2016,46.0,58618.0,0.851852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,251-300,Zhejiang University,China,44.3,19.6,46.0,36.1,96.2,-,47508.0,15.9,5%,41 : 59,2016,41.0,19478.0,0.694915
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,20%,46 : 54,2016,46.0,1104.0,0.851852
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,2218.0,8.0,14%,49 : 51,2016,49.0,1087.0,0.960784
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,18 : 82,2016,18.0,437.0,0.219512


In [63]:
times1 = times.groupby('country', as_index=False).sum()[['country','num_students']]
times1

,country,num_students
0,Argentina,108373.0
1,Australia,3082024.0
2,Austria,668005.0
3,Bangladesh,62716.0
4,Belarus,29303.0
...,...,...
67,Unisted States of America,0.0
68,United Arab Emirates,14447.0
69,United Kingdom,4926093.0
70,United States of America,15030134.0


In [64]:
df= pd.merge(university_ratio, times1, on = 'country')
df= df.rename(columns={"num_students_y":"tot_students_ country"})
df

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students_x,student_staff_ratio,international_students,female_male_ratio,year,female_%,female_stud,ratio,tot_students_ country
0,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,48.0,8363.0,0.923077,585033.0
1,201-250,Copenhagen Business School,Denmark,19.3,88.5,26.5,79.4,35.3,-,18293.0,42.7,19%,48 : 52,2016,48.0,8781.0,0.923077,585033.0
2,=167,Technical University of Denmark,Denmark,37.3,85.1,29.9,77.8,50.0,51.1,9990.0,5.0,18%,27 : 73,2016,27.0,2697.0,0.369863,585033.0
3,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,32.0,5152.0,0.470588,428705.0
4,501-600,Lappeenranta University of Technology,Finland,27.1,53.0,27.5,20.8,80.3,-,3486.0,23.9,19%,31 : 69,2016,31.0,1081.0,0.449275,428705.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,601-800,University of Ibadan,Nigeria,22.7,26.1,11.1,4.6,29.2,-,26247.0,17.1,1%,45 : 55,2016,45.0,11811.0,0.818182,26247.0
342,601-800,University of Indonesia,Indonesia,18.1,37.2,8.0,13.9,40.0,-,43565.0,15.8,0%,51 : 49,2016,51.0,22218.0,1.040816,43565.0
343,=193,University of Luxembourg,Luxembourg,25.0,99.8,26.7,84.8,38.1,49.4,5144.0,15.9,52%,50 : 50,2016,50.0,2572.0,1.000000,5144.0
344,601-800,University of Marrakech Cadi Ayyad,Morocco,17.4,42.7,6.3,12.1,29.2,-,56060.0,38.7,1%,46 : 54,2016,46.0,25788.0,0.851852,112120.0


In [65]:
df['fraction']= df['num_students_x']/df['tot_students_ country']*100
df

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students_x,student_staff_ratio,international_students,female_male_ratio,year,female_%,female_stud,ratio,tot_students_ country,fraction
0,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,48.0,8363.0,0.923077,585033.0,2.977952
1,201-250,Copenhagen Business School,Denmark,19.3,88.5,26.5,79.4,35.3,-,18293.0,42.7,19%,48 : 52,2016,48.0,8781.0,0.923077,585033.0,3.126832
2,=167,Technical University of Denmark,Denmark,37.3,85.1,29.9,77.8,50.0,51.1,9990.0,5.0,18%,27 : 73,2016,27.0,2697.0,0.369863,585033.0,1.707596
3,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,32.0,5152.0,0.470588,428705.0,3.755263
4,501-600,Lappeenranta University of Technology,Finland,27.1,53.0,27.5,20.8,80.3,-,3486.0,23.9,19%,31 : 69,2016,31.0,1081.0,0.449275,428705.0,0.813147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,601-800,University of Ibadan,Nigeria,22.7,26.1,11.1,4.6,29.2,-,26247.0,17.1,1%,45 : 55,2016,45.0,11811.0,0.818182,26247.0,100.000000
342,601-800,University of Indonesia,Indonesia,18.1,37.2,8.0,13.9,40.0,-,43565.0,15.8,0%,51 : 49,2016,51.0,22218.0,1.040816,43565.0,100.000000
343,=193,University of Luxembourg,Luxembourg,25.0,99.8,26.7,84.8,38.1,49.4,5144.0,15.9,52%,50 : 50,2016,50.0,2572.0,1.000000,5144.0,100.000000
344,601-800,University of Marrakech Cadi Ayyad,Morocco,17.4,42.7,6.3,12.1,29.2,-,56060.0,38.7,1%,46 : 54,2016,46.0,25788.0,0.851852,112120.0,50.000000


## 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [66]:
attainment.head(5)

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [67]:
df8 = attainment.dropna(subset= ['country_name', 'series_name'])

In [68]:
df8['country_name'].isnull().sum()

0

In [69]:
df8

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


## 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [70]:
anni = list(attainment.columns)
del anni[0:2]
anni

['1985',
 '1986',
 '1987',
 '1990',
 '1991',
 '1992',
 '1993',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2015']

In [71]:
df9 = pd.melt(df8, id_vars = ['country_name','series_name'], value_vars = anni, var_name = 'year').dropna(subset = ['year', 'value'])
df9

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03000
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83000
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54000
...,...,...,...,...
2054020,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.48356
2054021,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.88820
2054022,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000
2054023,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000


## 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [72]:
rank1 = pd.DataFrame(cwur.groupby('university_name').count()['year'])
rank1['cwur'] = 1
rank1

,year,cwur
university_name,,
AGH University of Science and Technology,2,1
Aalborg University,2,1
Aalto University,2,1
Aarhus University,2,1
Aberystwyth University,2,1
...,...,...
École centrale de Lyon,1,1
École normale supérieure - Paris,4,1
École normale supérieure de Cachan,1,1


In [73]:
rank2 = pd.DataFrame(shangai.groupby('university_name').count()['year'])
rank2['shangai'] = 1
rank2

,year,shangai
university_name,,
Aalborg University,2,1
Aalto University,7,1
Aarhus University,11,1
Aix Marseille University,3,1
Aix-Marseille University,1,1
...,...,...
Yamaguchi University,7,1
Yeshiva University,9,1
Yonsei University,9,1


In [74]:
rank3 = pd.DataFrame(times.groupby('university_name').count()['year'])
rank3['times'] = 1
rank3

,year,times
university_name,,
AGH University of Science and Technology,1,1
Aalborg University,5,1
Aalto University,5,1
Aarhus University,6,1
Aberystwyth University,5,1
...,...,...
École Normale Supérieure,6,1
École Normale Supérieure de Lyon,6,1
École Polytechnique,6,1


In [75]:
result = pd.concat([rank1, rank2, rank3],axis = 1, sort = True)
result = result.replace(np.nan, 0)
result

,year,cwur,year,shangai,year,times
AGH University of Science and Technology,2.0,1.0,0.0,0.0,1.0,1.0
Aalborg University,2.0,1.0,2.0,1.0,5.0,1.0
Aalto University,2.0,1.0,7.0,1.0,5.0,1.0
Aarhus University,2.0,1.0,11.0,1.0,6.0,1.0
Aberystwyth University,2.0,1.0,0.0,0.0,5.0,1.0
...,...,...,...,...,...,...
École centrale de Lyon,1.0,1.0,0.0,0.0,0.0,0.0
École normale supérieure - Paris,4.0,1.0,0.0,0.0,0.0,0.0
École normale supérieure de Cachan,1.0,1.0,0.0,0.0,0.0,0.0
École normale supérieure de Lyon,2.0,1.0,0.0,0.0,0.0,0.0


In [76]:
df10 = pd.DataFrame(columns = ['num_ranks'])
df10['num_ranks'] = result['cwur'] + result['shangai'] + result['times']
df10

,num_ranks
AGH University of Science and Technology,2.0
Aalborg University,3.0
Aalto University,3.0
Aarhus University,3.0
Aberystwyth University,2.0
...,...
École centrale de Lyon,1.0
École normale supérieure - Paris,1.0
École normale supérieure de Cachan,1.0
École normale supérieure de Lyon,1.0


## 11. In the times ranking, compute the number of times each university appears

In [85]:
dataframe11=pd.DataFrame(columns=['count'])
dataframe11['count']=times.groupby('university_name', as_index=True).size()
dataframe11

,count
university_name,
AGH University of Science and Technology,1
Aalborg University,5
Aalto University,5
Aarhus University,6
Aberystwyth University,5
...,...
École Normale Supérieure,6
École Normale Supérieure de Lyon,6
École Polytechnique,6


## 12. Find the universities that appear at most twice in the times ranking

In [90]:
dataframe12=dataframe11[dataframe11['count']>=2]
dataframe12

,count
university_name,
Aalborg University,5
Aalto University,5
Aarhus University,6
Aberystwyth University,5
Alexandria University,3
...,...
Zhejiang University,6
École Normale Supérieure,6
École Normale Supérieure de Lyon,6


## 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).


In [101]:
cwur['world_rank']=cwur['world_rank'].astype(str)
cwur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            2200 non-null   object 
 1   university_name       2200 non-null   object 
 2   country               2200 non-null   object 
 3   national_rank         2200 non-null   int64  
 4   quality_of_education  2200 non-null   int64  
 5   alumni_employment     2200 non-null   int64  
 6   quality_of_faculty    2200 non-null   int64  
 7   publications          2200 non-null   int64  
 8   influence             2200 non-null   int64  
 9   citations             2200 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2200 non-null   int64  
 12  score                 2200 non-null   float64
 13  year                  2200 non-null   int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 240.8+ KB


In [102]:
merged=pd.merge(times[['world_rank', 'university_name','year']],shangai[['world_rank', 'university_name','year']], on=['world_rank','university_name','year'])
merged

,world_rank,university_name,year
0,1,Harvard University,2011
1,2,Stanford University,2012
2,9,University of Chicago,2012
3,11,Yale University,2012
4,96,Michigan State University,2012
5,2,Stanford University,2013
6,11,Yale University,2013
7,15,University of Pennsylvania,2013
8,35,"University of California, Santa Barbara",2013
9,6,Princeton University,2014


In [103]:
merged1= pd.merge(merged,cwur[['world_rank','university_name','year']],on= ['world_rank','university_name','year'])
merged1

,world_rank,university_name,year
0,2,Stanford University,2013


### un'altra soluzione al quesito 13

In [62]:
md=pd.merge(times[['world_rank', 'university_name','year']],shangai[['world_rank', 'university_name','year']], on=['university_name'])
md

,world_rank_x,university_name,year_x,world_rank_y,year_y
0,1,Harvard University,2011,1,2005
1,1,Harvard University,2011,1,2006
2,1,Harvard University,2011,1,2007
3,1,Harvard University,2011,1,2008
4,1,Harvard University,2011,1,2009
...,...,...,...,...,...
13531,601-800,Texas Tech University,2016,303-401,2009
13532,601-800,Texas Tech University,2016,301-400,2010
13533,601-800,Texas Tech University,2016,401-500,2011
13534,601-800,Texas Tech University,2016,401-500,2015


In [63]:
md1= pd.merge(md,cwur[['world_rank','university_name','year']],on= ['university_name'])
md1.head(20)

,world_rank_x,university_name,year_x,world_rank_y,year_y,world_rank,year
0,1,Harvard University,2011,1,2005,1,2012
1,1,Harvard University,2011,1,2005,1,2013
2,1,Harvard University,2011,1,2005,1,2014
3,1,Harvard University,2011,1,2005,1,2015
4,1,Harvard University,2011,1,2006,1,2012
5,1,Harvard University,2011,1,2006,1,2013
6,1,Harvard University,2011,1,2006,1,2014
7,1,Harvard University,2011,1,2006,1,2015
8,1,Harvard University,2011,1,2007,1,2012
9,1,Harvard University,2011,1,2007,1,2013


In [93]:
md2 = md1[md1['world_rank_x']==md1['world_rank_y']]
md2

,world_rank_x,university_name,year_x,world_rank_y,year_y,world_rank,year
0,1,Harvard University,2011,1,2005,1,2012
1,1,Harvard University,2011,1,2005,1,2013
2,1,Harvard University,2011,1,2005,1,2014
3,1,Harvard University,2011,1,2005,1,2015
4,1,Harvard University,2011,1,2006,1,2012
...,...,...,...,...,...,...,...
32652,401-500,University of Siena,2016,401-500,2011,446,2015
32653,401-500,Xiamen University,2016,401-500,2010,354,2014
32654,401-500,Xiamen University,2016,401-500,2010,313,2015
32655,401-500,Xiamen University,2016,401-500,2011,354,2014


In [95]:
md3 = md2[md2['world_rank_y']==md2['world_rank']]
md3

,world_rank_x,university_name,year_x,world_rank_y,year_y,world_rank,year
0,1,Harvard University,2011,1,2005,1,2012
1,1,Harvard University,2011,1,2005,1,2013
2,1,Harvard University,2011,1,2005,1,2014
3,1,Harvard University,2011,1,2005,1,2015
4,1,Harvard University,2011,1,2006,1,2012
...,...,...,...,...,...,...,...
2513,9,University of Chicago,2014,9,2012,9,2013
2517,9,University of Chicago,2014,9,2013,9,2013
2521,9,University of Chicago,2014,9,2014,9,2013
2525,9,University of Chicago,2014,9,2015,9,2013


In [96]:
md4 = md3[md3['year_x']==md3['year_y']]
md4

,world_rank_x,university_name,year_x,world_rank_y,year_y,world_rank,year
24,1,Harvard University,2011,1,2011,1,2012
25,1,Harvard University,2011,1,2011,1,2013
26,1,Harvard University,2011,1,2011,1,2014
27,1,Harvard University,2011,1,2011,1,2015
601,2,Stanford University,2012,2,2012,2,2013
602,2,Stanford University,2012,2,2012,2,2014
603,2,Stanford University,2012,2,2012,2,2015
649,2,Stanford University,2013,2,2013,2,2013
650,2,Stanford University,2013,2,2013,2,2014
651,2,Stanford University,2013,2,2013,2,2015


In [97]:
md5 = md4[md4['year_y']==md4['year']]
md5

,world_rank_x,university_name,year_x,world_rank_y,year_y,world_rank,year
649,2,Stanford University,2013,2,2013,2,2013
